<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test-base-nvidia-Llama-3.1-Nemotron-Nano-8B-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
userdata.get('secretName')

In [ ]:
%pip install 'accelerate>=0.26.0'
%pip install torch
%pip install transformers
%pip install accelerate
%pip install -U transformers bitsandbytes
!python -m pip install optimum


In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")
print(f"Using device: {pipe.device}")


GPU available: True
Using device: cpu


In [1]:
import torch
from transformers import pipeline, AutoConfig
import json

# Explicitly set the `trust_remote_code=True` parameter in AutoConfig.from_pretrained:
# AutoConfig.from_pretrained(model, _from_pipeline=task, code_revision=code_revision, **hub_kwargs, **model_kwargs, trust_remote_code=True)

device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU

# Set `trust_remote_code=True` in the pipeline
pipe = pipeline(
    task="text-generation",
    model="nvidia/Llama-3.1-Nemotron-Nano-8B-v1",
    max_new_tokens=512,
    device=device,  # Automatically handles device placement
    torch_dtype=torch.float16 if device == 0 else None,  # Optimize for GPU
    model_kwargs={"use_cache": True},
    trust_remote_code=True  # Allow downloading and using remote code
)

pipe.model = pipe.model.to(torch.bfloat16)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token

def save_conversation(chat_history, filename="cooking_session.json"):
    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

def cook_recipe(recipe_data, automated_inputs):
    # Initialize fresh conversation
    chat = [
        {
            "role": "system",
            "content": "You are a master chef. Guide through recipe steps sequentially step-by-step. Start with asking if the user is ready to cook for proceeding to step 1 of the recipe provided and the subsequent steps."
        }
    ]

    # Structured recipe submission
    chat.append({
        "role": "user",
        "content": f"""Guide me through this recipe step-by-step:

        TITLE: {recipe_data['title']}

        INGREDIENTS:
        {chr(10).join(recipe_data['ingredients'])}

        INSTRUCTIONS:
        {recipe_data['instructions']}"""
    })

    # Process automated inputs
    for user_input in automated_inputs:
        # Generate response
        outputs = pipe(chat)
        chef_response = outputs[0]["generated_text"][-1]["content"]
        print(f"\nChef: {chef_response}")
        chat.append({"role": "system", "content": chef_response})

        # Add automated input
        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})

        if "exit" in user_input.lower():
            break

    save_conversation(chat)
    return chat

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
            """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "exit"
]

# Start cooking session with automated inputs
cook_recipe(recipe, automated_inputs)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Chef: It seems like you're going through a recipe step-by-step, but you haven't provided the instructions yet. Please continue by providing the next step or the first instruction you'd like to ask about. I'm here to help you with any questions or clarifications you might have as you proceed with the recipe.

Trainee: Done Continue.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Chef: It seems like you're repeatedly sending a "Done Continue" message, which suggests you'd like me to proceed with the recipe steps. However, I'll need a bit more context to understand where I should start. Please provide the following information to help me get started:

1. **Current Step**: Which step of the recipe are you ready to proceed with (e.g., Step 1, Step 2, etc.)?
2. **Previous Actions**: What have you done so far in the recipe? (e.g., prepared ingredients, started cooking, etc.)

With this information, I can guide you through the remaining steps of the recipe. If you're just starting and haven't completed any steps, I'll begin with Step 1.

Please provide the details, and I'll be happy to help you through the recipe.

Trainee: Done Continue.

Chef: It seems like you're going through a recipe step-by-step, but you haven't provided the current instruction or the point where you'd like me to continue. Could you please clarify which step or instruction you'd like me to foc

[{'role': 'system',
  'content': 'You are a master chef. Guide through recipe steps sequentially step-by-step. Start with asking if the user is ready to cook for proceeding to step 1 of the recipe provided and the subsequent steps.'},
 {'role': 'user',
  'content': 'Guide me through this recipe step-by-step:\n\n        TITLE: Miso-Butter Roast Chicken With Acorn Squash Panzanella\n\n        INGREDIENTS:\n        1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher salt, divided, plus more\n2 small acorn squash (about 3 lb. total)\n2 Tbsp. finely chopped sage\n1 Tbsp. finely chopped rosemary\n6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\n¼ tsp. ground allspice\nPinch of crushed red pepper flakes\nFreshly ground black pepper\n⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\n2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces\n2 Tbsp. extra-virgin olive oil\n½ small red onion, thinly sliced\n3 Tbsp. apple cider vin